In [36]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import t
from importlib import reload
import logging
reload(logging)
import funcs 
import pickle

In [37]:
# Задаем формат логирования
logging.basicConfig(
   format="%(levelname)s: %(asctime)s: %(message)s",
    level=logging.INFO
)

In [38]:
# Cоздаем лог-файл
logger = funcs.get_logger(path="logs/", file="forecast.log")

In [39]:
# Считаем файл-csv импортированный из БД мониторинга в датафрейм
file_name = 'api01.csv'
data = pd.read_csv(file_name, sep=' ')
logger.info(f"Data file: {file_name}")
logger.info(f"Data shape: {data.shape}")
logger.info(f"Data head: \n{data.head()}")

INFO: 2024-09-11 06:29:51,157: Data file: api01.csv
INFO: 2024-09-11 06:29:51,161: Data shape: (40109, 4)
INFO: 2024-09-11 06:29:51,179: Data head: 
         date      time          id  value
0  12.08.2024  14:32:24  1723462344    912
1  12.08.2024  14:30:40  1723462240    657
2  12.08.2024  14:29:17  1723462157    872
3  12.08.2024  14:28:14  1723462094    500
4  12.08.2024  14:27:04  1723462024    852


In [103]:
# Получим выборку с очищенными данными
data_cleaned = funcs.get_data_cleaned(data, logger=logger)

INFO: 2024-09-11 07:14:51,538: Number of entries before filling in gaps: 1440
INFO: 2024-09-11 07:14:51,588: Number of entries after filling in the blanks: 1440
INFO: 2024-09-11 07:14:51,642: Basic statistical characteristics of end-to-end values of a time series: 
        lower_bound       median  upper_bound
count  1440.000000  1440.000000  1440.000000
mean    300.369132   343.182986   478.380243
std      11.924069    31.102221   139.257811
min     255.800000   287.000000   314.850000
25%     293.000000   322.000000   368.000000
50%     301.000000   336.500000   429.875000
75%     307.900000   357.500000   559.625000
max     351.050000   511.000000  1541.250000
INFO: 2024-09-11 07:14:58,507: Transformed data: 
                       y
date_time               
2024-07-13 14:33:00  785
2024-07-13 14:34:00  547
2024-07-13 14:35:00  448
2024-07-13 14:36:00  415
2024-07-13 14:38:00  381
INFO: 2024-09-11 07:14:58,516: Number of records before filling gaps and merging duplicates: 40109
INFO

In [44]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('pipeline.pkl', 'rb') as pkl_file:
    pipe = pickle.load(pkl_file)

In [45]:
index_test = pd.DatetimeIndex(pd.date_range(data_cleaned.index.max() + pd.Timedelta(1, 'min'), periods=1440, freq='min'),
                name='date_time')
X = pd.DataFrame(index=index_test)

In [46]:
# Трансформируем исходные данные 
X_transformed = funcs.get_data_transformed(X)
logger.info('Shape before transform: {}'.format(X.shape))
logger.info('Shape after transform: {}'.format(X_transformed.shape))

INFO: 2024-09-11 06:30:43,259: Shape before transform: (1440, 0)
INFO: 2024-09-11 06:30:43,263: Shape after transform: (1440, 85)


In [106]:
# Сделаем предсказание обучающей выборки
y_pred = pipe.predict(X_transformed)

In [114]:
# Объявим вспомогательную функцию по вычислению остаточной стандартной ошибки: residual standard error
def get_rse(y_true, y_predicted):
    """
    - y_true: Actual values
    - y_predicted: Predicted values
    """
    y_true = np.array(y_true)
    n = y_true.shape[0]
    if n - 2 > 0:
        y_predicted = np.array(y_predicted)
        rss = np.sum(np.square(y_true - y_predicted))
        return np.sqrt(rss / (n - 2))
    else:
        return 0

In [105]:
# Зададим параметры вычисления 
prediction_level = 0.95
boundaries = round((1 - prediction_level) / 2, 2)
quantiles = [boundaries, prediction_level + boundaries]
# Зададим степень уверенности
gamma = prediction_level
# Уровень значимости
alpha = 1 - gamma
# Зададим продолжительность сезона в минутах
season = 1440
# Размер выборки
n = int(data_cleaned.shape[0] / season)
# Число степеней свободы
k = n - 1
# t-критическое
t_crit = -t.ppf(alpha/2, k)
# Объявим функцию для вычисления границы интервала прогнозирования для каждой минуты тестовой выборки
def get_prediction_interval(series):
  # Получим серию значений временного ряда за аналогичные минуты в пред. n суток
  y = data_cleaned.y[data_cleaned.index.time == datetime.time(series.name.hour, series.name.minute)]
  # Расчитаем отдельно верхную и нижнюю границы
  # Отфильтруем актуальные значения отдельно
  y_true_upper = y[y>series.y]
  y_true_lower = y[y<series.y]
  # Получим остаточную стандартную ошибку
  rse_upper = get_rse(y_true_upper, series.y)
  rse_lower = get_rse(y_true_lower, series.y)
  # погрешность
  eps_upper = t_crit * rse_upper
  eps_lower = t_crit * rse_lower
  # левая (нижняя) граница
  lower_bound = series.y - eps_lower
  # правая (верхняя) граница
  upper_bound = series.y + eps_upper
  return lower_bound, upper_bound

In [108]:
# Вычислим границы интервала прогнозирования для каждой минуты прогнозной выборки
# Cохраним результат в выборку для хранения прогноза
df_y_pred = pd.DataFrame({'y': y_pred}, index=X_transformed.index)
df_y_pred[['lower_bound', 'upper_bound']] = df_y_pred.apply(get_prediction_interval, axis=1).to_list()

In [112]:
logger.info(f'Main statistical characteristics of the model predictions: \n{df_y_pred.describe()}')

INFO: 2024-09-11 07:21:01,224: Main statistical characteristics of the model predictions: 
                 y  lower_bound  upper_bound
count  1440.000000  1440.000000  1440.000000
mean    335.484527   286.542925   439.187703
std      26.230752    20.610254   114.576162
min     285.725159   172.170042   304.012910
25%     317.118269   276.372262   356.633470
50%     328.786351   288.266261   395.861327
75%     349.429326   299.067340   497.667361
max     459.134506   344.729679  1298.835279


In [113]:
# Сохраним предсказания на сутки вперед в csv файл.
file_name = 'prediction.csv' 
df_y_pred.to_csv(file_name)
logger.info(f'Model predictions are saved to file: {file_name}')

INFO: 2024-09-11 07:22:05,116: Model predictions are saved to file: prediction.csv
